In [2]:
# import packages
import numpy as np
from pandas import read_csv
import pandas as pd
# import tensorflow as tf  # use the tf1 not tf2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt
import os
from absl import flags
from sklearn.metrics import confusion_matrix

# set learning rate
learning_rate = 0.0000000001

# set file path
dataset = "./epi_r.csv"
datapath = "documents/ml_tf/final_project"

# about data

## dataset 

* https://www.kaggle.com/hugodarwood/epirecipes?select=epi_r.csv  
Epicurious - Recipes with Rating and Nutrition
具有评级和营养的食谱

* explore different factors affecting people's enjoyment of food and/or cooking!
* 按食谱评级、营养信息和指定类别（稀疏）列出的超过 20k 食谱。我稍后可能会上传按配方创建日期分类的版本，还包括配方成分。
* Recipe information lifted from: http://www.epicurious.com/recipes-menus

## task
* predict if food is a dessert
* predict rating

## Load Data

In [3]:
df = pd.read_csv(datapath + dataset)

In [4]:
df

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 104.0+ MB


In [5]:
df.nunique()

title         17736
rating            8
calories       1858
protein         282
fat             326
              ...  
cookbooks         2
leftovers         2
snack             2
snack week        2
turkey            2
Length: 680, dtype: int64

In [8]:
df.rating.value_counts()

4.375    8019
3.750    5169
5.000    2719
0.000    1836
3.125    1489
2.500     532
1.250     164
1.875     124
Name: rating, dtype: int64

In [9]:
df.dessert.value_counts()

0.0    16479
1.0     3573
Name: dessert, dtype: int64

In [22]:
cols_sel = ['rating', 'dessert','calories', 'protein', 'fat', 'sodium',
              'peanut free', 'soy free','tree nut free','vegetarian','gourmet','kosher',
             'pescatarian','quick & easy','wheat/gluten-free','bake','summer','dairy free',
             'side','no sugar added','winter','fall','dinner','sugar conscious',
              'healthy','kidney friendly','onion', 'tomato', 'vegetable', 'milk/cream',
              'fruit', 'vegan', 'kid-friendly', 'egg', 'spring', 'herb',
              'garlic', 'salad', 'dairy', 'thanksgiving', 'appetizer', 'lunch', 
              'cheese', 'chicken', 'roast', 'no-cook', 'soup/stew', 'cocktail party',
              'ginger', 'potato', 'chill', 'grill/barbecue', 'lemon', 'drink', 
               'sauce', 'low cal', 'christmas', 'high fiber', 'food processor'
             ]
len(cols_sel)

59

In [23]:
df_extract = df[cols_sel]

In [24]:
df_extract.nunique()

rating                  8
dessert                 2
calories             1858
protein               282
fat                   326
sodium               2434
peanut free             2
soy free                2
tree nut free           2
vegetarian              2
gourmet                 2
kosher                  2
pescatarian             2
quick & easy            2
wheat/gluten-free       2
bake                    2
summer                  2
dairy free              2
side                    2
no sugar added          2
winter                  2
fall                    2
dinner                  2
sugar conscious         2
healthy                 2
kidney friendly         2
onion                   2
tomato                  2
vegetable               2
milk/cream              2
fruit                   2
vegan                   2
kid-friendly            2
egg                     2
spring                  2
herb                    2
garlic                  2
salad                   2
dairy       

In [25]:
label = df_extract['dessert'].to_numpy()
label, label.shape

(array([0., 0., 0., ..., 0., 0., 0.]), (20052,))

In [26]:
df_extract.shape, len(cols_sel[2:])

((20052, 59), 57)

In [27]:
features = df_extract[cols_sel[2:]].to_numpy()
features, features.shape

(array([[426.,  30.,   7., ...,   0.,   0.,   0.],
        [403.,  18.,  23., ...,   0.,   0.,   1.],
        [165.,   6.,   7., ...,   0.,   0.,   0.],
        ...,
        [563.,  31.,  38., ...,   0.,   0.,   0.],
        [631.,  45.,  24., ...,   0.,   0.,   0.],
        [560.,  73.,  10., ...,   1.,   0.,   0.]]),
 (20052, 57))

In [29]:
 df_extract[cols_sel[2:]].head()

,calories,protein,fat,sodium,peanut free,soy free,tree nut free,vegetarian,gourmet,kosher,...,potato,chill,grill/barbecue,lemon,drink,sauce,low cal,christmas,high fiber,food processor
0,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,547.0,20.0,32.0,452.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## split to train / test

In [35]:
model_dir = 'documents/ml_tf/final_project/fv4'

In [30]:
train_len = int(len(features) * 0.7)
train_label, train_data = label[:train_len], features[:train_len]
test_label, test_data = label[train_len:], features[train_len:]

# model 

## feature_columns

In [49]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[57])]

In [50]:
feature_columns[0]

NumericColumn(key='x', shape=(57,), default_value=None, dtype=tf.float32, normalizer_fn=None)

## LinearRegressor mode: 'wide'

In [66]:
classifier = tf.estimator.LinearRegressor (
    feature_columns=feature_columns, 
    model_dir=model_dir,
#     optimizer=lambda: tf.keras.optimizers.Ftrl(
#         learning_rate=tf.compat.v1.train.exponential_decay(
#             learning_rate=0.1,
#             global_step=tf.compat.v1.train.get_global_step(),
#             decay_steps=10000,
#             decay_rate=0.96))
)
classifier

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'documents/ml_tf/final_project/fv4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [73]:
train_label,train_data

(array([0., 0., 0., ..., 0., 0., 1.]),
 array([[426.,  30.,   7., ...,   0.,   0.,   0.],
        [403.,  18.,  23., ...,   0.,   0.,   1.],
        [165.,   6.,   7., ...,   0.,   0.,   0.],
        ...,
        [718.,  49.,  16., ...,   0.,   0.,   0.],
        [755., 117.,  26., ...,   0.,   0.,   1.],
        [317.,   1.,  11., ...,   0.,   0.,   0.]]))

In [69]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                x={"x": np.array(train_data)},
                y=np.array(train_label),
                num_epochs=None,
                shuffle=True)

In [71]:
classifier.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from documents/ml_tf/final_project/fv4\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 4000...
INFO:tensorflow:Saving checkpoints for 4000 into documents/ml_tf/final_project/fv4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 4000...
ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.

In [48]:
import tempfile
print(tempfile.gettempdir())

C:\Users\CHUANK~1.FAN\AppData\Local\Temp
